# ETL y EDA sobre el dataset de steam_games para el Proyecto ML Ops de Henry

## Problema de negocio:
Se solicita predecir el precio de un videojuego. 

## Objetivos:
Habiendo probado la primera limpieza de los datos (utilizada exitósamente sobre el main.py), debemos investigar las relaciones que hay entre las variables del dataset, comprobar si hay outliers o anomalías, y verificar patrones interesantes que valgan la pena explorar para crear un modelo predictivo eficiente. Este deberá basarse en características como Género, Año, Metascore o cualquiera de aquellas que resulten adecuadas.

### Describimos las variables que conforman el dataframe

* publisher: Empresa publicadora del contenido
* genres: Género del contenido
* app_name: Nombre del contenido
* title: Título del contenido
* url: URL de publicación del contenido
* release_date: Fecha de lanzamiento
* tags: Etiquetas de contenido
* discount_price: Precio de descuento
* reviews_url: Reviews de contenido
* specs: Especificaciones
* price: Precio del contenido
* early_access: Acceso temprano
* id: Identificador único de contenido
* developer: Desarrollador
* sentiment: Análisis de sentimientos
* metascore: Score por metacritic

### Generación del primer Data Wrangling.

In [ ]:
# Importamos de las librerías necesarias
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")

In [ ]:
# Recuperamos los datos desde el archivo .json provisto
dataset = []
with open('dataset/steam_games.json') as f:
    dataset.extend(ast.literal_eval(line) for line in f)
    
# Creamos el dataframe a partir del dataset obtenido
data = pd.DataFrame(dataset)

In [ ]:
# Verificamos la estructura de nuestro dataframe
data.shape

In [ ]:
# Verificamos cómo está conformado
data.info()

In [ ]:
# Hacemos una comprobación de los valores nulos
data.isnull().sum()

In [ ]:
# Reubicamos la variable 'id' para un mejor lectura
cols = list(data.columns)
cols.remove('id')
cols = ['id'] + cols
data = data[cols]

In [ ]:
# Verificamos si la variable 'id' contiene nulos
filas_null = data[data['id'].isna()]
filas_null

In [ ]:
# Identificamos las filas duplicadas en la variable 'id'
filas_dup = data[data.duplicated('id', keep=False)]
filas_dup

In [ ]:
# Eliminamos las filas con nulos y duplicadas en la variable 'id'
data_con_valores_nuevos = data.copy()
data_con_valores_nuevos.drop([74, 14573], inplace=True)
data_con_valores_nuevos.reset_index(drop=True, inplace=True)

In [ ]:
# Adecuamos el valor nulo en la variable 'id' en correlación a los demás valores existentes
filas_con_nulos = data_con_valores_nuevos[data_con_valores_nuevos['id'].isnull()].index
valores_no_nulos_ordenados = data_con_valores_nuevos.dropna(subset=['id']).sort_values('id')['id'].unique()
data_con_valores_nuevos.loc[filas_con_nulos, 'id'] = valores_no_nulos_ordenados[:len(filas_con_nulos)]

In [ ]:
# Verificamos nuestro nuevo orden de variables y comprobamos la composición de nuestro dataframe
data_con_valores_nuevos

* Hasta aquí nos hemos asegurado de que el dataframe inicial contenga la información adecuada para comenzar con un análisis sin escollos. Hemos visto que unas de las variables de interés, 'metascore', del total de filas del dataframe, contiene casi un 95% de valores nulos, y que la variable 'id' contenía valores duplicados y nulos, que se han corregido y a su vez fue recolocada para poder utilizarla como identificador único en el caso de querer relacionar la presente información con otros datasets en un análisis ulterior.   

### Distribución de los datos

In [ ]:
# Adecuación y limpieza del dataframe
data_steam = data_con_valores_nuevos.copy()
data_steam['release_date'] = pd.to_datetime(data_steam['release_date'], errors='coerce')
data_steam['metascore'] = pd.to_numeric(data_steam['metascore'], errors='coerce')
data_steam['price'] = pd.to_numeric(data_steam['price'], errors='coerce')
reemplazar_valores = {'publisher': '', 'genres': '', 'tags': '', 'discount_price': 0,
                      'specs': '', 'reviews_url': '', 'app_name': '', 'title': '',
                       'id': '', 'sentiment': '', 'developer': ''}
data_steam.fillna(value=reemplazar_valores, inplace=True)
data_steam = data_steam.dropna(subset=['price'])
data_steam = data_steam.dropna(subset=['release_date'])
data_steam = data_steam.dropna(subset=['metascore'])
data_steam.reset_index(drop=True, inplace=True)

In [ ]:
# Inspeccionamos la variable 'price', que más tarde usaremos en nuestro modelo predictor
data_steam['price'].unique()

In [ ]:
# Hacemos limpieza de las variables numéricas
data_steam['price'] = data_steam['price'].astype(float)
data_steam['metascore'] = data_steam['metascore'].replace('NA', np.nan)
data_steam = data_steam.dropna(subset=['metascore'])
data_steam['metascore'] = data_steam['metascore'].astype(int)

In [ ]:
# Definimos una función para extraer los datos de las variables que contienen listas
def extraer_listas(df, columns_to_explode):
    data_exploded = df.copy()
    for column in columns_to_explode:
        data_exploded = data_exploded.explode(column)
    data_exploded.reset_index(drop=True, inplace=True)
    return data_exploded

In [ ]:
# Seleccionamos y explotamos las variables de interés
col_listas = ['genres', 'tags', 'specs']
data_steam_final = extraer_listas(data_steam, col_listas)

In [ ]:
# Contamos los valores únicos por variable
data_steam_final.nunique()

In [ ]:
# Mostramos el dataframe final
data_steam_final.info()

* Con la vista puesta en normalizar los datos entre las distintas variables, se decidió eliminar los datos nulos o faltantes, los cuales eran una constante en todo el dataframe, adecuando también el tipo de dato correcto para cada columna. Asmimismo, se entrajeron los datos contenidos dentro de listas en las variables 'genres', 'specs' y 'tags'. Luego, llegamos al resultado de emparejar la información en todas las variables, logrando que todos los valores coincidan sin nulos o faltantes. 

### Análisis de Componentes Principales (PCA).

In [ ]:
# Importación de las librerías necesarias
import category_encoders as ce
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
data_steam_metascore = data_steam_final.copy()
# Codificamos las variables categóricas a numéricas para una mejor utilización del modelo
a_numérica = ce.OrdinalEncoder(cols=['id', 'publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'discount_price', 'reviews_url', 'specs', 'price',
       'early_access', 'developer', 'sentiment', 'metascore'])

# Definimos un dataframe para el cálculo del PCA 
data_steam_pca = a_numérica.fit_transform(data_steam_metascore)
data_steam_pca

In [ ]:
# Entrenamos el modelo PCA con escalado de los datos y lo agrupamos en un pipeline
pipe_pca = make_pipeline(MinMaxScaler(), PCA())
pipe_pca.fit(data_steam_pca)
modelo_pca = pipe_pca.named_steps['pca']

In [ ]:
# Calculamos el porcentaje de varianza explicada acumulada
prop_varianza_acum = modelo_pca.explained_variance_ratio_.cumsum()
print(prop_varianza_acum)

# Lo mostramos en un gráfico
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.plot(np.arange(len(data_steam_pca.columns)) + 1, prop_varianza_acum, marker = 'o')
for x, y in zip(np.arange(len(data_steam_pca.columns)) + 1, prop_varianza_acum):
    label = round(y, 2)
    ax.annotate(label, (x,y), textcoords="offset points", xytext=(0,10), ha='center')
ax.set_ylim(0, 1.1)
ax.set_xticks(np.arange(modelo_pca.n_components_) + 1)
ax.set_title('Porcentaje de Varianza Explicada Acumulada')
ax.set_xlabel('Componente Principal')
ax.set_ylabel('% Varianza Acumulada')

In [ ]:
# Definimos ahora un nuevo dataframe para mostrar el trabajo del PCA en columnas
pca_df = data_steam_pca
pca_df2 = pca_df
pca_df = MinMaxScaler().fit_transform(pca_df)
pca_df = pd.DataFrame(pca_df,columns=pca_df2.columns).set_index(pca_df2.index)
pca_df

In [ ]:
# Procedemos a aplicar el PCA y calcular los porcentajes en las muestras explicadas anteriormente
pca = PCA(n_components=16)
principalComponents = pca.fit_transform(pca_df)
principalComp_Df = pd.DataFrame(data = principalComponents, columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10','pca11','pca12','pca13','pca14','pca15','pca16']).set_index(pca_df.index)
print("Forma del dataframe de los Componentes Principales", principalComp_Df.shape)
explicacion = pca.explained_variance_ratio_
print(explicacion)
print('suma:', sum(explicacion[:16]))

In [ ]:
# Mostramos el dataframe de PCA logrado
principalComp_Df

* Este somero análisis de componentes principales, en la idea de que al reducir la dimensionalidad de nuestro dataframe a un 80/85 porciento de explicación con el mínimo de columnas posibles es suficiente a los fines de nuestro mejor análisis de datos, nos lleva a concluir que acotando nuestro dataframe principal a 6 columnas, tendríamos suficientes componentes principales para llevar el estudio de nuestros datos al éxito.

### Análisis de los datos y visualizaciones

In [ ]:
# Reducimos las variables del dataframe a las de interés
steam_eda_reduc = data_steam_final.drop(['id', 'publisher', 'app_name', 'url', 'title', 'reviews_url', 'sentiment'], axis=1, inplace=True)
steam_eda_reduc = data_steam_final[data_steam_final['price'] != 0]
steam_eda_reduc['genres'] = steam_eda_reduc['genres'].replace('', np.nan)
steam_eda_reduc = steam_eda_reduc.dropna(subset=['genres'])
steam_eda_reduc.reset_index(drop=True, inplace=True)
steam_eda_reduc.columns

##### Nube de palabras para las variables no numéricas

In [ ]:
# Creamos los objetos WordCloud para las variables 'genres', 'tags', 'specs' y 'developer'
steam_eda_reduc['genres'] = steam_eda_reduc['genres'].astype(str)
all_genres = ','.join(steam_eda_reduc['genres']).lower()
wordcloud_genres = WordCloud(width=1200, height=600, background_color='black', colormap='viridis', max_words=100).generate(all_genres)
steam_eda_reduc['tags'] = steam_eda_reduc['tags'].astype(str)
all_tags = ','.join(steam_eda_reduc['tags']).lower()
wordcloud_tags = WordCloud(width=1200, height=600, background_color='black', colormap='viridis', max_words=100).generate(all_tags)
steam_eda_reduc['specs'] = steam_eda_reduc['specs'].astype(str)
all_specs = ','.join(steam_eda_reduc['specs']).lower()
wordcloud_specs = WordCloud(width=1200, height=600, background_color='black', colormap='viridis', max_words=100).generate(all_specs)
steam_eda_reduc['developer'] = steam_eda_reduc['developer'].astype(str)
all_developers = ','.join(steam_eda_reduc['developer']).lower()
wordcloud_developers = WordCloud(width=1200, height=600, background_color='black', colormap='viridis', max_words=100).generate(all_developers)
# Definimos subplots con 2 filas y 2 columnas para las nubes de palabras
plt.figure(figsize=(15, 12))
# Subplot 1: Géneros
plt.subplot(2, 2, 1)
plt.imshow(wordcloud_genres, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Géneros')
# Subplot 2: Tags
plt.subplot(2, 2, 2)
plt.imshow(wordcloud_tags, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Etiquetas')
# Subplot 3: Specs
plt.subplot(2, 2, 3)
plt.imshow(wordcloud_specs, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Especificaciones')
# Subplot 4: Developers
plt.subplot(2, 2, 4)
plt.imshow(wordcloud_developers, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Desarrolladores')
# Ajustamos los subplots para evitar superposición de etiquetas
plt.tight_layout()
# Mostramos el gráfico
plt.show()


* Nótese que entre géneros y etiquetas, Indie y Action mantienen una notable similitud en el primer y segundo puesto, ya no hallándose el mismo patrón para los demás. Ya respecto a las especificaciones, single, player y steam marcan el top 3, mientras que para desarrolladores, al parecer interactive es la palabra más elegida entre ellos.

##### Matriz de correlación entre la variable 'genres' y las demás numéricas

In [ ]:
# Definimos una copia del dataframe principal
steam_eda_corr = steam_eda_reduc.copy()
# Codificamos las variables categóricas utilizando one-hot encoding
steam_eda_corr = pd.get_dummies(steam_eda_corr, columns=['genres'])
# Calculamos la matriz de correlación
correlation_matrix = steam_eda_corr.corr()
# Creamos la visualización de la matriz de correlación
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

* Interesante es darnos cuenta, luego de analizar la presente matriz, que la variable 'early_access' comparte una correlación por debajo del 0.5 respecto del género denominado (uno pensaría de inmediato que debería ser una correlación perfecta) Early Access, sobresaliendo luego las correlaciones -aunque bajas, presentes- entre los géneros de acción e indie, acción y aventuras e indie y aventuras.

##### Crosstab para las variables 'genres' y 'early_access'

In [ ]:
# Creamos un dataframe para revisión
pd.crosstab(index=steam_eda_reduc['genres'], columns=steam_eda_reduc['early_access'])

In [ ]:
# Creamos la visualización de las variables detalladas
pd.crosstab(index=steam_eda_reduc['genres'], columns=steam_eda_reduc['early_access']).plot.barh(stacked=True, figsize=(20, 15))
plt.show()

* Sin más, al parecer la plataforma de juegos no es partidaria de accesos tempranos en casi ninguno de sus géneros.

##### Catplot para las variables 'genres' y 'release_date'

In [ ]:
# Creamos el catplot para las variables detalladas
plt.figure(figsize=(12, 12))
sns.catplot(x='genres', y='release_date', data=steam_eda_reduc, jitter=0.5, hue='genres', palette='Set1', height=10, aspect=2)
plt.xticks(rotation=90)
plt.xlabel('Género')
plt.ylabel('Fecha de lanzamiento')
plt.title('Género en relación a su fecha de lanzamiento')
plt.show()

* Podemos observar que los géneros casual y aventuras, fueron los primeros en lanzarse, que entre los años 2008 y 2017 se concentran la mayor cantidad de lanzamientos y que Free to Play y Early Access, se pelean por ser el género menos lanzado.

##### Violinplot para las variables 'genres', 'metascore' y 'early_access'

In [ ]:
# Usamos un tipo de gráfico boxplot para mostrar los resultados del 'describe' 
plt.figure(figsize=(10, 15))
sns.violinplot(data=steam_eda_reduc, x='genres', y='metascore', hue='early_access')
plt.xticks(rotation=90)
plt.show()

* Al contrario de lo que venimos viendo, resulta que el género Acceso Temprano como así el tipo de acceso temprano, aquí batallan mano a mano en cuanto a su metascore, respecto de géneros como acción e indie, con una concentración cercana al puntaje 87, teniendo a su vez como el género con más variación de puntajes a Action.

##### Histograma para las variables 'price' y 'metascore'

In [ ]:
# Inicializamos con una comprobación de la distribución en un pequeño dataframe
steam_eda_reduc[['price', 'metascore']].describe()

In [ ]:
# Nos aseguramos que las variables no contienen valores no numéricos
steam_eda_reduc = steam_eda_reduc.dropna(subset=['price', 'metascore'])
# Convertir las columnas 'price' y 'metascore' al tipo de datos numérico si es necesario (omitir si ya son numéricas)
steam_eda_reduc['price'] = pd.to_numeric(steam_eda_reduc['price'], errors='coerce')
steam_eda_reduc['metascore'] = pd.to_numeric(steam_eda_reduc['metascore'], errors='coerce')
# Creamos una figura con 1 fila y 2 columnas para los subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
# Histograma de 'price' en el primer subplot
sns.histplot(steam_eda_reduc['price'], bins=100, kde=False, color='gray', ax=ax2)
min_price = steam_eda_reduc['price'].min()
max_price = steam_eda_reduc['price'].max()
ax2.set_xlabel('Precio')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Histograma del Precio de los Videojuegos')
ax2.text(0.75, 0.9, f"Media: {steam_eda_reduc['price'].mean():.2f}\nMediana: {steam_eda_reduc['price'].median():.2f}\nDesviación Estándar: {steam_eda_reduc['price'].std():.2f}",
         transform=ax2.transAxes)
ax2.axvline(min_price, color='red', linestyle='dashed', linewidth=2, label=f'Mínimo: {min_price:.2f}')
ax2.axvline(max_price, color='green', linestyle='dashed', linewidth=2, label=f'Máximo: {max_price:.2f}')
ax2.legend()
# Histograma de 'metascore' en el segundo subplot
sns.histplot(steam_eda_reduc['metascore'], bins=100, kde=False, color='gray', ax=ax1)
min_metascore = steam_eda_reduc['metascore'].min()
max_metascore = steam_eda_reduc['metascore'].max()
ax1.set_xlabel('Metascore')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Histograma del Metascore de los Videojuegos')
ax1.text(0.75, 0.9, f"Media: {steam_eda_reduc['metascore'].mean():.2f}\nMediana: {steam_eda_reduc['metascore'].median():.2f}\nDesviación Estándar: {steam_eda_reduc['metascore'].std():.2f}",
         transform=ax1.transAxes)
ax1.axvline(min_metascore, color='red', linestyle='dashed', linewidth=2, label=f'Mínimo: {min_metascore:.2f}')
ax1.axvline(max_metascore, color='green', linestyle='dashed', linewidth=2, label=f'Máximo: {max_metascore:.2f}')
ax1.legend()
# Ajustamos los subplots para evitar superposición de etiquetas
plt.tight_layout()
# Mostramos el gráfico
plt.show()

* Ambos histogramas resultan muy claros a la lectura, no obstante ser notorios como la mayoria de los gaps en la variables metascore parecieran aparecen cada tres marcas de puntajes.

##### Scatterplot para las variables 'price' y 'metascore'

In [ ]:
# Creamos el scatter plot para las variables de interés
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_steam, x='metascore', y='price', size='price', sizes=(30, 120), palette='viridis', alpha=0.7, legend=False)
plt.xlabel('Metascore')
plt.ylabel('Precio')
plt.title('Scatter Plot: Metascore vs Precio')
# Agregamos una la línea de varianza
reg_plot = sns.regplot(data=data_steam, x='metascore', y='price', scatter=False, color='red')
# Obtenenemos los resultados de la regresión lineal
x = data_steam['metascore']
y = data_steam['price']
model = sns.regplot(data=data_steam, x='metascore', y='price', scatter=False, color='red')
slope, intercept = model.get_lines()[0].get_data()
# Imprimimos los resultados de la regresión lineal
print(f"Coeficiente de la línea de regresión (pendiente): {slope[1]:.2f}")
print(f"Término independiente de la línea de regresión (intercepto): {intercept[1]:.2f}")
# Mostramos el gráfico
plt.show()

* Con lo visualizado, podemos concluir que para cada aumento de una unidad en el metascore, el precio del juego aumenta en promedio en 20.77 unidades, teniendo como punto de partida que cuando el metascore sea cero, el precio sería de aproximadamente 6.86 unidades.

##### Boxplot entre las variables 'price', 'metascore' y 'early_access'

In [ ]:
# Creamos el boxplot de las variables de interés  
plt.figure(figsize=(10, 15))
sns.boxplot(data=steam_eda_reduc, x='price', y='metascore', hue='early_access')
plt.xticks(rotation=90)
plt.show()

* Claro resulta, luego del boxplot, que se presentan valores únicos y atípicos, en general, en unidades de metascore por debajo de 60 en la mayoría de las unidades de precio, manteniéndose un constante volumen de puntajes entre los 65 y 80 puntos.

##### Boxplot entre las variables 'genres', 'price' y 'early_access'

In [ ]:
# Creamos el boxplot de las variables de interés  
plt.figure(figsize=(10, 15))
sns.boxplot(data=steam_eda_reduc, x='genres', y='price', hue='early_access')
plt.xticks(rotation=90)
plt.show()

* Es interesante ver, en este segundo boxplot que, al comparar los géneros con el precio, la visualización se invierte -en relación al boxplot anterior- casi a un 100%. Aquí observamos que los valores únicos o atípicos se dan por encima de los 20 puntos de precio, generándose la mayor concentración entre los 10 y los 25. Como punto de atención, es llamativo ver que en los géneros Free To Play, el volumen de precios es más elevado que en los demás géneros. 